This script loads behavioral mice data (from `biasedChoiceWorld` protocol and, separately, the last three sessions of training) only from mice that pass a given (stricter) training criterion. For the `biasedChoiceWorld` protocol, only sessions achieving the `trained_1b` and `ready4ephysrig` training status are collected.
The data are slightly reformatted and saved as `.csv` files.

In [1]:
import datajoint as dj
dj.config['database.host'] = 'datajoint.internationalbrainlab.org'

from ibl_pipeline import subject, acquisition, action, behavior, reference, data
from ibl_pipeline.analyses.behavior import PsychResults, SessionTrainingStatus
from ibl_pipeline.utils import psychofit as psy
from ibl_pipeline.analyses import behavior as behavior_analysis

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Connecting lacerbi@datajoint.internationalbrainlab.org:3306


In [2]:
import os
myPath = r"C:\Users\Luigi\Documents\GitHub\ibl-changepoint\data" # Write here your data path
os.chdir(myPath)

In [3]:
# Get list of mice that satisfy given training criteria (stringent trained_1b)
# Check query from behavioral paper: 
# https://github.com/int-brain-lab/paper-behavior/blob/master/paper_behavior_functions.py

subj_query = (subject.Subject * subject.SubjectLab * reference.Lab * subject.SubjectProject
                  & 'subject_project = "ibl_neuropixel_brainwide_01"').aggr(
                          (acquisition.Session * behavior_analysis.SessionTrainingStatus())
#                          & 'training_status="trained_1a" OR training_status="trained_1b"',
#                          & 'training_status="trained_1b" OR training_status="ready4ephysrig"',
                          & 'training_status="trained_1b"',
                          'subject_nickname', 'sex', 'subject_birth_date', 'institution',
                          date_trained='min(date(session_start_time))')

subjects = (subj_query & 'date_trained < "2019-10-31"' & 'date_trained >= "2019-07-10"')

mice_names = sorted(subjects.fetch('subject_nickname'))

print(mice_names)

['CSHL_014', 'CSHL_015', 'CSH_ZAD_001', 'CSH_ZAD_003', 'CSH_ZAD_004', 'CSH_ZAD_006', 'CSH_ZAD_007', 'CSH_ZAD_010', 'KS004', 'KS014', 'KS015', 'KS016', 'KS017', 'KS019', 'KS022', 'KS023', 'KS024', 'KS025', 'SWC_009', 'SWC_013', 'SWC_014', 'SWC_021', 'SWC_022', 'SWC_023', 'ZM_1743', 'ibl_witten_12', 'ibl_witten_14', 'ibl_witten_15', 'ibl_witten_16']


In [4]:
sess_train  = ((acquisition.Session * behavior_analysis.SessionTrainingStatus) & 
               'task_protocol LIKE "%training%"' & 'session_start_time < "2019-09-30"')
sess_stable = ((acquisition.Session * behavior_analysis.SessionTrainingStatus) & 
               'task_protocol LIKE "%biased%"' & 'session_start_time < "2019-09-30"' & 
               ('training_status="trained_1b" OR training_status="ready4ephysrig"'))
              
stable_mice_names = list()

# Perform at least this number of sessions
MinSessionNumber = 4

def get_mouse_data(df):
    position_deg = 35.    # Stimuli appear at +/- 35 degrees

    # Create new dataframe
    datamat = pd.DataFrame()
    datamat['trial_num'] = df['trial_id']
    datamat['session_num'] = np.cumsum(df['trial_id'] == 1)
    datamat['stim_probability_left'] = df['trial_stim_prob_left']
    signed_contrast = df['trial_stim_contrast_right'] - df['trial_stim_contrast_left']
    datamat['contrast'] = np.abs(signed_contrast)
    datamat['position'] = np.sign(signed_contrast)*position_deg
    datamat['response_choice'] = df['trial_response_choice']
    datamat.loc[df['trial_response_choice'] == 'CCW','response_choice'] = 1
    datamat.loc[df['trial_response_choice'] == 'CW','response_choice'] = -1
    datamat.loc[df['trial_response_choice'] == 'No Go','response_choice'] = 0
    datamat['trial_correct'] = np.double(df['trial_feedback_type']==1)
    datamat['reaction_time'] = df['trial_response_time'] - df['trial_stim_on_time'] # double-check

    # Since some trials have zero contrast, need to compute the alleged position separately
    datamat.loc[(datamat['trial_correct'] == 1) & (signed_contrast == 0),'position'] = \
        datamat.loc[(datamat['trial_correct'] == 1) & (signed_contrast == 0),'response_choice']*position_deg
    datamat.loc[(datamat['trial_correct'] == 0) & (signed_contrast == 0),'position'] = \
        datamat.loc[(datamat['trial_correct'] == 0) & (signed_contrast == 0),'response_choice']*(-position_deg)
    
    return datamat

In [ ]:
# Loop over all mice
for mouse_nickname in mice_names:
    
    mouse_subject = {'subject_nickname': mouse_nickname}
    
    # Get mouse data for biased sessions
    behavior_stable = (behavior.TrialSet.Trial & (subject.Subject & mouse_subject)) \
        * sess_stable.proj('session_uuid','task_protocol','session_start_time','training_status') * subject.Subject.proj('subject_nickname') \
        * subject.SubjectLab.proj('lab_name')
    df = pd.DataFrame(behavior_stable.fetch(order_by='subject_nickname, session_start_time, trial_id', as_dict=True))

    if len(df) > 0:           # The mouse has performed in at least one stable session with biased blocks            
        datamat = get_mouse_data(df)

        # Take mice that have performed a minimum number of sessions
        if np.max(datamat['session_num']) >= MinSessionNumber:

            # Should add 'N' to mice names that start with numbers?

            # Save dataframe to CSV file
            filename = mouse_nickname + '.csv'
            datamat.to_csv(filename,index=False)
            stable_mice_names.append(mouse_nickname)
            
            # Get mouse last sessions of training data
            behavior_train = (behavior.TrialSet.Trial & (subject.Subject & mouse_subject)) \
                * sess_train.proj('session_uuid','task_protocol','session_start_time') * subject.Subject.proj('subject_nickname') \
                * subject.SubjectLab.proj('lab_name')
            df_train = pd.DataFrame(behavior_train.fetch(order_by='subject_nickname, session_start_time, trial_id', as_dict=True))

            datamat_train = get_mouse_data(df_train)
            Nlast = np.max(datamat_train['session_num']) - 3
            datamat_final = datamat_train[datamat_train['session_num'] > Nlast]

            # Save final training dataframe to CSV file
            filename = mouse_nickname + '_endtrain.csv'
            datamat_final.to_csv(filename,index=False)
            

print(stable_mice_names)

In [19]:
ss = (((acquisition.Session * behavior_analysis.SessionTrainingStatus) & 
               'task_protocol LIKE "%biased%"' & 'session_start_time < "2019-09-30"' & 
               ('training_status="trained_1b" OR training_status="ready4ephysrig"')) * subject.Subject) & ('subject_nickname = "ibl_witten_07"')

In [20]:
ss

subject_uuid,session_start_time start time,session_uuid,session_number number,session_end_time end time,session_lab name of lab,session_location name of the location,task_protocol,session_type type,session_narrative,session_ts,training_status,subject_nickname nickname,sex sex,subject_birth_date birth date,ear_mark ear mark,subject_line name,subject_source name of source,protocol_number protocol number,subject_description,subject_ts
02120449-9b19-4276-a434-513886c2fb19,2019-07-18 12:03:27,8a0cf27e-0044-4e9b-9967-4bc9ece3589b,1,2019-07-18 13:12:22,wittenlab,_iblrig_wittenlab_behavior_1,_iblrig_tasks_biasedChoiceWorld5.1.2,Experiment,None,2019-08-06 21:42:14,ready4ephysrig,ibl_witten_07,F,2018-11-13,None,C57BL/6J,Jax,3,None,2019-09-25 01:33:25
02120449-9b19-4276-a434-513886c2fb19,2019-07-19 13:01:38,180a2e18-c4fe-4f4d-92a2-4ff7e56ea669,1,2019-07-19 14:20:33,wittenlab,_iblrig_wittenlab_behavior_1,_iblrig_tasks_biasedChoiceWorld5.2.0,Experiment,None,2019-08-06 21:42:14,ready4ephysrig,ibl_witten_07,F,2018-11-13,None,C57BL/6J,Jax,3,None,2019-09-25 01:33:25
02120449-9b19-4276-a434-513886c2fb19,2019-07-22 16:46:19,84a43b9d-43d4-436c-82f1-dd804867c50c,1,2019-07-22 18:11:50,wittenlab,_iblrig_wittenlab_behavior_1,_iblrig_tasks_biasedChoiceWorld5.2.1,Experiment,None,2019-08-06 21:42:14,ready4ephysrig,ibl_witten_07,F,2018-11-13,None,C57BL/6J,Jax,3,None,2019-09-25 01:33:25
02120449-9b19-4276-a434-513886c2fb19,2019-07-23 11:37:38,539f6608-9ff5-41e1-b0ad-4480bdb54f15,1,2019-07-23 13:06:49,wittenlab,_iblrig_wittenlab_behavior_1,_iblrig_tasks_biasedChoiceWorld5.2.2,Experiment,None,2019-08-06 21:42:14,ready4ephysrig,ibl_witten_07,F,2018-11-13,None,C57BL/6J,Jax,3,None,2019-09-25 01:33:25
02120449-9b19-4276-a434-513886c2fb19,2019-07-24 12:17:33,ed763960-490d-4490-87d3-38be2985e217,1,2019-07-24 13:48:47,wittenlab,_iblrig_wittenlab_behavior_1,_iblrig_tasks_biasedChoiceWorld5.2.2,Experiment,None,2019-08-06 21:42:14,ready4ephysrig,ibl_witten_07,F,2018-11-13,None,C57BL/6J,Jax,3,None,2019-09-25 01:33:25
02120449-9b19-4276-a434-513886c2fb19,2019-07-25 11:51:05,fd8f339d-55d8-4aa5-a4c3-5d8f1eae852e,1,2019-07-25 13:13:11,wittenlab,_iblrig_wittenlab_behavior_1,_iblrig_tasks_biasedChoiceWorld5.2.3,Experiment,None,2019-08-06 21:42:14,ready4ephysrig,ibl_witten_07,F,2018-11-13,None,C57BL/6J,Jax,3,None,2019-09-25 01:33:25
02120449-9b19-4276-a434-513886c2fb19,2019-07-26 11:10:48,0e8fe973-05d0-4dfc-809e-fc48a9810bbd,1,2019-07-26 12:42:27,wittenlab,_iblrig_wittenlab_behavior_1,_iblrig_tasks_biasedChoiceWorld5.2.3,Experiment,None,2019-08-06 21:42:14,ready4ephysrig,ibl_witten_07,F,2018-11-13,None,C57BL/6J,Jax,3,None,2019-09-25 01:33:25
02120449-9b19-4276-a434-513886c2fb19,2019-07-29 12:51:10,61a88005-d3ff-4d9f-9615-ecf5777f8721,2,2019-07-29 14:21:42,wittenlab,_iblrig_wittenlab_behavior_1,_iblrig_tasks_biasedChoiceWorld5.2.5,Experiment,None,2019-08-06 21:42:14,ready4ephysrig,ibl_witten_07,F,2018-11-13,None,C57BL/6J,Jax,3,None,2019-09-25 01:33:25
02120449-9b19-4276-a434-513886c2fb19,2019-07-30 10:18:04,411f83e6-7251-435f-880b-2c4c0991f187,1,2019-07-30 11:26:11,wittenlab,_iblrig_wittenlab_behavior_1,_iblrig_tasks_biasedChoiceWorld5.2.5,Experiment,None,2019-08-06 21:42:14,ready4ephysrig,ibl_witten_07,F,2018-11-13,None,C57BL/6J,Jax,3,None,2019-09-25 01:33:25
02120449-9b19-4276-a434-513886c2fb19,2019-07-31 10:41:45,34273b65-92d6-49f6-8a28-434733f61e38,1,2019-07-31 12:03:21,wittenlab,_iblrig_wittenlab_behavior_1,_iblrig_tasks_biasedChoiceWorld5.2.5,Experiment,None,2019-08-06 21:42:14,ready4ephysrig,ibl_witten_07,F,2018-11-13,None,C57BL/6J,Jax,3,None,2019-09-25 01:33:25


In [8]:
mice_names

['CSHL_014',
 'CSHL_015',
 'CSH_ZAD_001',
 'CSH_ZAD_003',
 'CSH_ZAD_004',
 'CSH_ZAD_006',
 'CSH_ZAD_007',
 'CSH_ZAD_010',
 'KS004',
 'KS014',
 'KS015',
 'KS016',
 'KS017',
 'KS019',
 'KS022',
 'KS023',
 'KS024',
 'KS025',
 'SWC_009',
 'SWC_013',
 'SWC_014',
 'SWC_021',
 'SWC_022',
 'SWC_023',
 'ZM_1743',
 'ibl_witten_12',
 'ibl_witten_14',
 'ibl_witten_15',
 'ibl_witten_16']

In [9]:
mouse_subject = {'subject_nickname': 'CSHL_014'}
    
# Get mouse data for biased sessions
behavior_stable = (behavior.TrialSet.Trial & (subject.Subject & mouse_subject)) \
    * sess_stable.proj('session_uuid','task_protocol','session_start_time','training_status') * subject.Subject.proj('subject_nickname') \
    * subject.SubjectLab.proj('lab_name')
df = pd.DataFrame(behavior_stable.fetch(order_by='subject_nickname, session_start_time, trial_id', as_dict=True))


In [12]:
list(df.columns)

['subject_uuid',
 'session_start_time',
 'trial_id',
 'trial_start_time',
 'trial_end_time',
 'trial_response_time',
 'trial_response_choice',
 'trial_stim_on_time',
 'trial_stim_contrast_left',
 'trial_stim_contrast_right',
 'trial_go_cue_time',
 'trial_go_cue_trigger_time',
 'trial_feedback_time',
 'trial_feedback_type',
 'trial_rep_num',
 'trial_stim_prob_left',
 'trial_reward_volume',
 'trial_iti_duration',
 'trial_included',
 'session_uuid',
 'task_protocol',
 'training_status',
 'subject_nickname',
 'lab_name']